In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# Define transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalization
])

# Load training and test datasets
batch_size = 128
trainset = torchvision.datasets.CIFAR10(root='./Work/data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./Work/data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Define class names
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Check dataset
dataiter = iter(trainloader)
images, labels = next(dataiter)
print(f"Batch shape: {images.shape}, Labels shape: {labels.shape}")


Batch shape: torch.Size([128, 3, 32, 32]), Labels shape: torch.Size([128])


In [11]:
import torch.nn.functional as F

class DeepCNN(nn.Module):
    def __init__(self):
        super(DeepCNN, self).__init__()

        # Convolutional Layers with Batch Normalization
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)  # Output: (64, 32, 32)
        self.bn1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Output: (128, 32, 32)
        self.bn2 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Reduces spatial size by half

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # Output: (256, 16, 16)
        self.bn3 = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)  # Output: (256, 16, 16)
        self.bn4 = nn.BatchNorm2d(256)

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)  # Output: (512, 8, 8)
        self.bn5 = nn.BatchNorm2d(512)

        self.conv6 = nn.Conv2d(512, 512, kernel_size=3, padding=1)  # Output: (512, 8, 8)
        self.bn6 = nn.BatchNorm2d(512)

        # Fully Connected Layers
        self.fc1 = nn.Linear(512 * 2 * 2, 1024)  # Corrected size based on final feature map
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)  # 10 classes in CIFAR-10

        self.dropout = nn.Dropout(0.5)  # Dropout to prevent overfitting

    def forward(self, x):
        # Convolutional Blocks
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # (64, 16, 16)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # (128, 8, 8)

        x = F.relu(self.bn3(self.conv3(x)))             # (256, 8, 8)
        x = self.pool(F.relu(self.bn4(self.conv4(x))))  # (256, 4, 4)

        x = F.relu(self.bn5(self.conv5(x)))             # (512, 4, 4)
        x = self.pool(F.relu(self.bn6(self.conv6(x))))  # (512, 2, 2)

        # Flatten for Fully Connected Layers
        x = x.view(x.size(0), -1)  # Convert to (batch_size, 2048)

        # Fully Connected Layers with Dropout
        x = self.dropout(F.relu(self.fc1(x)))  # (batch_size, 1024)
        x = self.dropout(F.relu(self.fc2(x)))  # (batch_size, 512)
        x = self.fc3(x)  # (batch_size, 10)

        return x


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DeepCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

In [13]:
import torch

x = torch.tensor([[1, 2, 3], [4, 5, 6]])
print("Original Shape:", x.shape)

# Reshaping using view()
y = x.view(3, 2)
print("Reshaped Tensor:\n", y)
print("New Shape:", y.shape)


Original Shape: torch.Size([2, 3])
Reshaped Tensor:
 tensor([[1, 2],
        [3, 4],
        [5, 6]])
New Shape: torch.Size([3, 2])


In [14]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
y = x.view(-1, 2)  # Automatically infer one dimension
print(y)


tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [ ]:
def train_model(model, trainloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        print(f"Epoch {epoch+1}: Loss: {running_loss/len(trainloader):.4f}, Accuracy: {100 * correct / total:.2f}%")

train_model(model, trainloader, criterion, optimizer, epochs=10)


In [16]:
def test_model(model, testloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

test_model(model, testloader)


Test Accuracy: 73.02%


In [17]:
from safetensors.torch import save_file

model_static = model.state_dict()
save_file(model_static, './Models/DeepCNNCiraf10.safetensors')
print('Models has sucessfully Saved!')

Models has sucessfully Saved!
